# Quicksight Migration 

### How to Migrate Quicksight Artefacts between two accounts/environments
**https://vipulbhatia.medium.com/aws-quicksight-migration-between-accounts-workflow-3b543cbc6d95**

### Instruction Manual

1. Proceed each step in the sequence mentioned in this notebook.
2. Get the programmatic access for the accounts and update the variables in cell#2 for Source Account and cell#3 for Target Account.
3. Set the Variables in cell#6 for the analysis and dashboard being migrated.
4. This notebook enables the migration for Analyses, Dashboards, and their corresponding datasets.
5. The pre-requisite for the dashboard is availability of a json formatted file that has cross-reference for the datasources in teh form of datasource Ids for each environment as stated in the example: {"dsn": [{"name": "Athena", "prod": "cXXXXXXX-fXXX-4XXX-8XXX-6XXXXXXXXX", "test": "cXXXXXXX-fXXX-4XXX-8XXX-6XXXXXXXXX", "dev": "cXXXXXXX-fXXX-4XXX-8XXX-6XXXXXXXXX"}]}
The lookup is based on the name and the items present with the keys ('prod', 'test' ,'dev') prod: Production, test: Test, dev: Development

## Begin Process Execution

In [ ]:
import os
import boto3
import io
import gc
import awswrangler as wr
import sys
import time
import json

## 1- Initialize the variables and Instantiate the QuickSight Clients using boto3

### Generate the instances for resource session and client

In [ ]:
source_session =  boto3.Session(profile_name= 'analytics-test', region_name='ap-southeast-2')
client = source_session.client('quicksight')

# Configure destination client
tgt_session =  boto3.Session(profile_name= 'analytics-dev', region_name='ap-southeast-2')
tgt_client = tgt_session.client('quicksight')




### Set the variables for the source and target

In [ ]:
v_src_account_id='888142126615'
v_analysis_id='07f040fd-6648-48e5-971c-fe04962874ce'
v_tgt_account_id='892988355045'
v_src_env='test'
v_target_env='dev'
v_role='AWSReservedSSO_AWSAdministratorAccess_09b685eec4a90b99'
v_user='PA.285833D@curtin.edu.au'


## 2- Analysis

### Extract the source analysis properties based on the analysis id set in the variables

In [ ]:
"""
Get the source Analysis ID using analysis name
"""
analysisId=''
analysisArn=''
analysisName=''
analyses = client.list_analyses(
    AwsAccountId=v_src_account_id,
    MaxResults=100
)
v_analysis = []
y = [(member['Arn'],member['AnalysisId'],member['Name'], member['Status'],member['CreatedTime'],member['LastUpdatedTime']) for member in analyses["AnalysisSummaryList"] ]
v_analysis.extend(y)
while 'NextToken' in analyses:
        analyses = client.list_analyses(
            AwsAccountId=v_src_account_id,
            MaxResults=100,
            NextToken=analyses['NextToken']
            )
        y = [(member['Arn'],member['AnalysisId'],member['Name'] if ('Name' in member.keys()) else "", member['Status'],member['CreatedTime'],member['LastUpdatedTime']) for member in analyses["AnalysisSummaryList"] ]
        v_analysis.extend(y)
        #i+=len(analyses.get('AnalysisSummaryList'))
#print("Number of Analyses: " + str(i))
for analysis in v_analysis:    
    if analysis[1]==v_analysis_id and analysis[3].find('_SUCCESSFUL')!=-1 and analysis[3].find('DELETE')==-1:
        analysisId=analysis[1]
        analysisName=analysis[2]
        analysisArn=analysis[0]
        break
        
src_analysisId=analysisId
src_analysis_arn=analysisArn
src_analysis_name=analysisName

In [ ]:
print("src_analysis_id is: {id}\nsrc_analysis_arn is: {arn}\nsrc_analysis_name is: {name}".format(id = src_analysisId,arn=src_analysis_arn,name=src_analysis_name))


In [ ]:
"""
Get the target Analysis ID using analysis name
"""
analysisId=''
analysisArn=''
analysisName=''

tgt_analysisId=''
tgt_analysis_arn=''
tgt_analysis_name=''

analyses = tgt_client.list_analyses(
    AwsAccountId=v_tgt_account_id,
    MaxResults=100
)
v_analysis = []
y = [(member['Arn'],member['AnalysisId'],member['Name'], member['Status'],member['CreatedTime'],member['LastUpdatedTime']) for member in analyses["AnalysisSummaryList"] ]
v_analysis.extend(y)
while 'NextToken' in analyses:
        analyses = tgt_client.list_analyses(
            AwsAccountId=v_tgt_account_id,
            MaxResults=100,
            NextToken=analyses['NextToken']
            )
        y = [(member['Arn'],member['AnalysisId'],member['Name'] if ('Name' in member.keys()) else "", member['Status'],member['CreatedTime'],member['LastUpdatedTime']) for member in analyses["AnalysisSummaryList"] ]
        v_analysis.extend(y)
        #i+=len(analyses.get('AnalysisSummaryList'))
#print("Number of Analyses: " + str(i))
for analysis in v_analysis:    
    if analysis[1]==v_analysis_id and analysis[3].find('_SUCCESSFUL')!=-1 and analysis[3].find('DELETE')==-1:
        analysisId=analysis[1]
        analysisArn=analysis[0]
        analysisName=analysis[2]
        break


if analysisId:
    tgt_analysisId=analysisId
    tgt_analysis_arn=analysisArn
    tgt_analysis_name=analysisName

In [ ]:
print("tgt_analysisId is: {id}\ntgt_analysis_arn is: {arn}\ntgt_analysis_name is: {name}".format(id = tgt_analysisId,arn=tgt_analysis_arn,name=tgt_analysis_name))  

### Set the Template Name

In [ ]:
v_template_name=src_analysisId.replace(" ","")+"_MigrationTemplate"

### Get the list of Datasets attached to the analysis

In [ ]:
###Get the description of analysis. This will tell us all the datasets that the analysis use
src_analysis_desc=client.describe_analysis(
    AwsAccountId=v_src_account_id,
    AnalysisId=src_analysisId
)

In [ ]:
print(src_analysis_desc)

### Get the IDs for all the datasets and store in a dictionary

In [ ]:
###Extract the dataset ARNs and Ids in 2 dictionaries.
v_src_DatasetArn_Dict={}
v_src_DatasetArn_Id={}
v_src_DatasetName={}

datasets = client.list_data_sets(
    AwsAccountId=v_src_account_id,
    MaxResults=100
)  

v_datasets = []
y = [(member['Arn'],member['DataSetId'],member['Name'], member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
v_datasets.extend(y)
while 'NextToken' in datasets:
        datasets = client.list_data_sets(
            AwsAccountId=v_src_account_id,
            MaxResults=100,
            NextToken=datasets['NextToken']
            )
        y = [(member['Arn'],member['DataSetId'],member['Name'] if ('Name' in member.keys()) else "", member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
        v_datasets.extend(y)


for src_analysis_dataset in src_analysis_desc['Analysis']['DataSetArns']:
    for dataset in v_datasets:    
        if dataset[0]==src_analysis_dataset:
            v_src_DatasetArn_Dict[src_analysis_dataset]=dataset[0]
            v_src_DatasetArn_Id[src_analysis_dataset]=dataset[1]
            v_src_DatasetName[src_analysis_dataset]=dataset[2]

In [ ]:
print(v_src_DatasetArn_Dict)
print(v_src_DatasetArn_Id)
print(v_src_DatasetName)

In [ ]:
"""
##convert this into a loop. we can have multiple datasets in an analysis
datasets = client.list_data_sets(
    AwsAccountId=v_src_account_id,
    MaxResults=100
)  

v_datasets = []
y = [(member['Arn'],member['DataSetId'],member['Name'], member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
v_datasets.extend(y)
while 'NextToken' in datasets:
        datasets = client.list_data_sets(
            AwsAccountId=v_src_account_id,
            MaxResults=100,
            NextToken=datasets['NextToken']
            )
        y = [(member['Arn'],member['DataSetId'],member['Name'] if ('Name' in member.keys()) else "", member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
        v_datasets.extend(y)
        #i+=len(datasets.get('DataSetSummaries'))
#print("Number of Analyses: " + str(i))
for dataset in v_datasets:    
    if dataset[0]==analysis['Analysis']['DataSetArns'][0]:
        datasetArn=dataset[0]
        datasetId=dataset[1]
        break
"""        

### Generate the unique ARNs datasets and store in a list. These ARNs will then be used to create the template for the target

In [ ]:
src_dataset_ARN_list=[]
n=0
for item in v_src_DatasetArn_Dict:
    n=n+1
    ##dict(DataSetPlaceholder=v_analysis_name.replace(" ","")+"_MigrationTemplateARN"+str(n),DataSetArn=item)
    src_dataset_ARN_list.append(dict(DataSetPlaceholder=src_analysisId.replace(" ","")+"_MigrationTemplateARN"+str(n),DataSetArn=item))


In [ ]:
print(src_dataset_ARN_list)

## 3- Create the Template

###  Check if the template already exists. If the template already exists, update it, else create it

In [ ]:
templateArn=''
templateId=''
templateName=''

templates = client.list_templates(
    AwsAccountId=v_src_account_id,
    MaxResults=100
)


v_templates = []
y = [(member['Arn'],member['TemplateId'],member['Name'], member['LatestVersionNumber'],member['CreatedTime'],member['LastUpdatedTime']) for member in templates["TemplateSummaryList"] ]
v_templates.extend(y)
while 'NextToken' in templates:
        templates = client.list_templates(
            AwsAccountId=v_src_account_id,
            MaxResults=100,
            NextToken=templates['NextToken']
            )
        y = [(member['Arn'],member['TemplateId'],member['Name'] if ('Name' in member.keys()) else "", member['LatestVersionNumber'],member['CreatedTime'],member['LastUpdatedTime']) for member in templates["TemplateSummaryList"] ]
        v_templates.extend(y)
        #i+=len(templates.get('TemplateSummaryList'))
#print("Number of Analyses: " + str(i))
for template in v_templates:    
    if template[1]==v_template_name:
        templateArn=template[0]
        templateId=template[1]
        templateName=template[2]             
        break
        
if templateArn:
    print("updating template")
    updateTemplate = client.update_template(
        AwsAccountId=v_src_account_id,
        TemplateId=v_template_name,
        SourceEntity={
            'SourceAnalysis': {
                'Arn': src_analysis_arn,
                'DataSetReferences': src_dataset_ARN_list
            }
        }
    )
else:
    print("creating template")
    createTemplate=client.create_template(
        AwsAccountId=v_src_account_id,
        TemplateId=v_template_name,
        Name= src_analysis_name, 
        SourceEntity={
            'SourceAnalysis': {
                'Arn': src_analysis_arn,
                'DataSetReferences': src_dataset_ARN_list
            }
        }
    )   

In [ ]:
try:
    print("CreateTemplateARN: "+createTemplate['Arn'])
except:
    print("UpdateTemplateARN: "+updateTemplate['Arn'])

In [ ]:
###Backup
"""
createTemplate=client.create_template(
    AwsAccountId='462393762422',
    TemplateId='ProductionOverviewTemplate_Migration',
    SourceEntity={
        'SourceAnalysis': {
            'Arn': analysisArn,
            'DataSetReferences': [
                {
                    'DataSetPlaceholder': 'ProductionOverview_DSPH_Migration',
                    'DataSetArn': datasetArn
                },
            ]
        }
    }
)"""

### Confirm if the template has been created. You can modify template permissions only if it has been successfully created.

Template permissions should be updated on the source account template to provide access to the Target account

In [ ]:
#Use this to check if the template has been created successfully. If yes, then move to the next step of listing and updating permissions
template_desc=client.describe_template(
    AwsAccountId=v_src_account_id,
    TemplateId=v_template_name,
    #VersionNumber=123,
    #AliasName='string'
)

if template_desc['Template']['Version']['Status'].find('_SUCCESSFUL')==-1:
    time.sleep(120)
    template_desc=client.describe_template(
        AwsAccountId=v_src_account_id,
        TemplateId=v_template_name,
        #VersionNumber=123,
        #AliasName='string'
    )    
    if template_desc['Template']['Version']['Status'].find('_SUCCESSFUL')==-1:
        print("Template could not be successfully created. Please check the configurations and re-execute the steps")
    else:
        print("Template was succefully created. Proceeding with the next step of updating permissions.")
        updateTemplate = client.update_template_permissions(
            AwsAccountId=v_src_account_id,
            TemplateId=v_template_name,
            GrantPermissions=[
                {
                    "Principal": "arn:aws:iam::"+v_tgt_account_id+":root",
                    'Actions': ["quicksight:UpdateTemplatePermissions","quicksight:DescribeTemplate"]
                },
            ]
        )
else:
    updateTemplate = client.update_template_permissions(
    AwsAccountId=v_src_account_id,
    TemplateId=v_template_name,
    GrantPermissions=[
        {
            "Principal": "arn:aws:iam::"+v_tgt_account_id+":root",
            'Actions': ["quicksight:UpdateTemplatePermissions","quicksight:DescribeTemplate"]
        },
    ]
)

    

#dict_keys(['Arn', 'Version', 'TemplateId', 'LastUpdatedTime', 'CreatedTime'])

In [ ]:
## Validate the response. Ensure that the template permissions have been updated in "Permissions" node.
print(updateTemplate['Permissions'])

In [ ]:
####We can remove this part
for key in v_src_DatasetArn_Id:
    
    dict_physical_table_map=client.describe_data_set(
        AwsAccountId=v_src_account_id,
        DataSetId=v_src_DatasetArn_Id[key])['DataSet']['PhysicalTableMap']

    distinct_dsn = set()

    for i in dict_physical_table_map.keys():
        for j in dict_physical_table_map[i].keys():
            for k in dict_physical_table_map[i][j].keys():
                if k=='DataSourceArn':
                    distinct_dsn.add(dict_physical_table_map[i][j]['DataSourceArn'])


    distinct_source_dsn_set=set()
    distinct_source_dsnid_set=set()
    distinct_target_dsn_set=set()


    for dsn in distinct_dsn:

        datasources = client.list_data_sources(
            AwsAccountId=v_src_account_id, ##Replace this with Target Account
            MaxResults=100
        )

        v_datasources = []
        y = [(member['Arn'],member['DataSourceId'],member['Name'], member['CreatedTime'],member['LastUpdatedTime'],member['Type']) for member in datasources["DataSources"] ]
        v_datasources.extend(y)
        while 'NextToken' in datasources:
                datasources = client.list_data_sources(
                    AwsAccountId=v_src_account_id,
                    MaxResults=100,
                    NextToken=datasources['NextToken']
                    )
                y = [(member['Arn'],member['DataSourceId'],member['Name'] if ('Name' in member.keys()) else "",member['CreatedTime'],member['LastUpdatedTime']) for member in datasources["DataSources"] ]
                v_datasources.extend(y)
                #i+=len(templates.get('TemplateSummaryList'))
        #print("Number of Analyses: " + str(i))
        for datasource in v_datasources:    
            """
            if datasource[2]=='Athena' :
                sourceDataSourceARN=datasource[0]
                sourceDataSourceId=datasource[1]
            """
            if datasource[0]==dsn:
                sourceDataSourceARN=datasource[0]
                sourceDataSourceId=datasource[1]
                sourceDataSourceName=datasource[2]
                distinct_source_dsn_set.add(sourceDataSourceId)

## 4- Data Source and Data set

### Update the Dataset Definition for each of the datasets. Create/Update the Datasets

Each dataset has physical table map, which has datasource arn attached to it. There can be multiple physical table maps and multiple data sources. We have a cross-reference json list of the mapping of all datasources for all the environments (accounts). At run time, we refer to this json file and extract the datasource arn's based on the environments and replace the data source arn in the physical table map section of the dataset definition. This is done inside of a loop "for k in v_src_DatasetARn_Id".
The loop iterates through the dataset definition for each dataset and at the end of each iteration, creates the dataset in the target account. If the dataset already exists, it updates it.
Each iteration also updates the permissions of the dataset, so it can be accessed via quicksight UI.

In [ ]:
get_dsn_mapping_dict=dict()

with open('data.json') as json_file:
    dsn_list=json.load(json_file)
    
for i in dsn_list['dsn']:
    get_dsn_mapping_dict[i[v_src_env]]=i[v_target_env]

n=0    
for k in v_src_DatasetArn_Id:
    
    
    update_flag='N'
    ds=client.describe_data_set(
            AwsAccountId=v_src_account_id,
            DataSetId=v_src_DatasetArn_Id[k])

    item_list=[]
    for item in ds['DataSet']['PhysicalTableMap']:
        item_list.append(item)

    for i in item_list:
        for key in ds['DataSet']['PhysicalTableMap'][i]:
            temp_string=ds['DataSet']['PhysicalTableMap'][i][key]['DataSourceArn'].replace(v_src_account_id,v_tgt_account_id)
            replacement_dsn=get_dsn_mapping_dict[temp_string[temp_string.find('/')+1:]]
            if not replacement_dsn:
                print("raise issue")
            else:
                temp_string=temp_string.replace(temp_string[temp_string.find('/')+1:],replacement_dsn)
                ds['DataSet']['PhysicalTableMap'][i][key]['DataSourceArn']=temp_string
                
                
    datasets = tgt_client.list_data_sets(
        AwsAccountId=v_tgt_account_id,
        MaxResults=100
    )  

    v_datasets = []
    y = [(member['Arn'],member['DataSetId'],member['Name'], member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
    v_datasets.extend(y)
    while 'NextToken' in datasets:
            datasets = tgt_client.list_data_sets(
                AwsAccountId=v_tgt_account_id,
                MaxResults=100,
                NextToken=datasets['NextToken']
                )
            y = [(member['Arn'],member['DataSetId'],member['Name'] if ('Name' in member.keys()) else "", member['CreatedTime'],member['LastUpdatedTime']) for member in datasets["DataSetSummaries"] ]
            v_datasets.extend(y)
    for dataset in v_datasets:    
        if dataset[1]==ds['DataSet']['DataSetId']:
            print('dataset already exists. updating the dataset instead of creating.')
            ###run update here
            update_flag='Y'
            print("In Update")
            target_dataset = tgt_client.update_data_set(
                AwsAccountId=v_tgt_account_id,
                DataSetId=ds['DataSet']['DataSetId'],
                Name=ds['DataSet']['Name'],
                PhysicalTableMap=ds['DataSet']['PhysicalTableMap'],
                LogicalTableMap=ds['DataSet']['LogicalTableMap'],
                ImportMode=ds['DataSet']['ImportMode'])
            break

    #print(update_flag)
    
    if update_flag=='N':
        print('dataset does not exists. creating the dataset')
        target_dataset = tgt_client.create_data_set(AwsAccountId=v_tgt_account_id, 
                                                    DataSetId=ds['DataSet']['DataSetId'],
                                                    Name=ds['DataSet']['Name'],
                                                    PhysicalTableMap=ds['DataSet']['PhysicalTableMap'],
                                                    LogicalTableMap=ds['DataSet']['LogicalTableMap'],
                                                    ImportMode=ds['DataSet']['ImportMode'])                           
        

    tgt_client.update_data_set_permissions(
        AwsAccountId=v_tgt_account_id,
        DataSetId=ds['DataSet']['DataSetId'],
        GrantPermissions=[
            {
                'Principal': 'arn:aws:quicksight:ap-southeast-2:'+v_tgt_account_id+':user/default/'+v_role+'/'+v_user,
                'Actions': [
                    'quicksight:UpdateDataSetPermissions','quicksight:DescribeDataSetPermissions','quicksight:PassDataSet','quicksight:CreateIngestion','quicksight:DescribeIngestion','quicksight:CancelIngestion','quicksight:ListIngestions','quicksight:UpdateDataSet','quicksight:DescribeDataSet','quicksight:DeleteDataSet'
                ]
            },
        ]
    ) 

## 5- Create or Update Analysis

The analysis requires Source Entity, which is obtained by modifying the definition of the dataset ARN in the src_dataset_ARN_list and storing it in the tgt_dataset_ARN_list.
Check if the analysis already exists in the target account. If it does, update it, else create it. Use the template ARN for the source template, for which we modified teh permissions in the Template step.
Update the permissions on the analysis, so it can be accessed from Quicksight.

In [ ]:
temp_tgt_dict=dict()
tgt_dataset_ARN_List=[]
for i in src_dataset_ARN_list:
    temp_tgt_dict['DataSetPlaceholder']=i['DataSetPlaceholder']
    temp_tgt_dict['DataSetArn']=i['DataSetArn'].replace(v_src_account_id,v_tgt_account_id)
    tgt_dataset_ARN_List.append(temp_tgt_dict)



if tgt_analysisId:
    print('Analysis exists in Target. Executing the steps for Update.')
    ###call update analysis function here. we can use the template for source to update the analysis with tgt_analysisId
    tgt_client.update_analysis(
                AwsAccountId=v_tgt_account_id,
                AnalysisId=src_analysisId,
                Name=tgt_analysis_name,
                SourceEntity={
                    'SourceTemplate': {
                        'DataSetReferences': tgt_dataset_ARN_List,
                        'Arn': template_desc['Template']['Arn']
                    }
                }
            )        
else:
    print('Analysis does not exist in Target. Executing the steps for Analysis creation.')
    ###call create analysis function here. we can potentially use the src_analysisId to store as the analysis ID during report creation
    tgt_client.create_analysis(
        AwsAccountId=v_tgt_account_id,
        AnalysisId=src_analysisId,
        Name=src_analysis_name,
        SourceEntity={
            'SourceTemplate': {
                'DataSetReferences': tgt_dataset_ARN_List,
                'Arn': template_desc['Template']['Arn']
            }
        }
    )

tgt_analysis_desc=tgt_client.describe_analysis(
    AwsAccountId=v_tgt_account_id,
    AnalysisId=src_analysisId
)

if tgt_analysis_desc['Analysis']['Status'].find('_SUCCESSFUL')==-1:
    time.sleep(120)
    tgt_analysis_desc=tgt_client.describe_analysis(
        AwsAccountId=v_tgt_account_id,
        AnalysisId=src_analysisId
    )
    if tgt_analysis_desc['Analysis']['Status'].find('_SUCCESSFUL')==-1:
        print("Analysis could not be successfully created/updated. Please check the configurations and re-execute the steps")
    else:
        print("Analysis was succefully created/updated. Proceeding with the next step of updating permissions.")
        tgt_client.update_analysis_permissions(
            AwsAccountId=v_tgt_account_id,
            AnalysisId=src_analysisId,
            GrantPermissions=[
                {
                    'Principal': 'arn:aws:quicksight:ap-southeast-2:'+v_tgt_account_id+':user/default/'+v_role+'/'+v_user,
                    'Actions': ['quicksight:RestoreAnalysis', 'quicksight:UpdateAnalysisPermissions', 'quicksight:DeleteAnalysis', 'quicksight:QueryAnalysis', 'quicksight:DescribeAnalysisPermissions', 'quicksight:DescribeAnalysis', 'quicksight:UpdateAnalysis']
                },
            ]
        )
else:
        print("Analysis was succefully created/updated. Proceeding with the next step of updating permissions.")
        tgt_client.update_analysis_permissions(
            AwsAccountId=v_tgt_account_id,
            AnalysisId=src_analysisId,
            GrantPermissions=[
                {
                    'Principal': 'arn:aws:quicksight:ap-southeast-2:'+v_tgt_account_id+':user/default/'+v_role+'/'+v_user,
                    'Actions': ['quicksight:RestoreAnalysis', 'quicksight:UpdateAnalysisPermissions', 'quicksight:DeleteAnalysis', 'quicksight:QueryAnalysis', 'quicksight:DescribeAnalysisPermissions', 'quicksight:DescribeAnalysis', 'quicksight:UpdateAnalysis']
                },
            ]
        ) 